# Medical transcript classification [sklearn]
* Multiclass classification of medical transcript.
* Reference notebook: <https://www.kaggle.com/code/leekahwin/text-classification-using-n-gram-0-8-f1/notebook>
* Dataset: <https://www.kaggle.com/code/leekahwin/text-classification-using-n-gram-0-8-f1/input>

## Install necessary dependencies

In [14]:
!pip install nltk

## Import libraries

In [15]:
import os
import string
from typing import Iterable

import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer

import giskard
from giskard import Dataset, Model

## Download NLTK stopwords corpus

In [16]:
# Download list of english stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mykytaalekseiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Define constants

In [17]:
# Constants.
LABELS_LIST = [
    'Neurosurgery',
    'ENT - Otolaryngology',
    'Discharge Summary',
    'General Medicine',
    'Gastroenterology',
    'Neurology',
    'SOAP / Chart / Progress Notes',
    'Obstetrics / Gynecology',
    'Urology'
]

TEXT_COLUMN_NAME = "transcription"
TARGET_COLUMN_NAME = "medical_specialty"

RANDOM_SEED = 8888

# Paths.
PATH_DATA = os.path.join(".", "datasets", "medical_transcript_classification_dataset", "mtsamples.csv")

## Load data

In [18]:
def load_data() -> pd.DataFrame:
    """Load and initially preprocess data."""
    df = pd.read_csv(PATH_DATA)

    # Drop useless columns.
    df = df.drop(columns=['Unnamed: 0', "description", "sample_name", "keywords"])

    # Trim text.
    df = df.apply(lambda x: x.str.strip())

    # Filter samples by label.
    df = df[df[TARGET_COLUMN_NAME].isin(LABELS_LIST)]

    # Drop rows with no transcript.
    df = df[df[TEXT_COLUMN_NAME].notna()]

    return df

transcript_df = load_data()

## Train-test split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(transcript_df[[TEXT_COLUMN_NAME]], transcript_df[TARGET_COLUMN_NAME],
                                                    random_state=RANDOM_SEED)

## Wrap dataset with giskard

In [20]:
raw_data = pd.concat([X_test, y_test], axis=1)
wrapped_data = Dataset(raw_data,
                       name="medical_transcript_dataset",
                       target=TARGET_COLUMN_NAME,
                       column_types={TEXT_COLUMN_NAME: "text"})

## Define preprocessing steps

In [21]:
stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")

def preprocess_text(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess text."""
    # Lower.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.lower())

    # Remove punctuation.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

    # Tokenize.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: x.split())

    # Stem.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: [stemmer.stem(word) for word in x])

    # Remove stop-words.
    df[TEXT_COLUMN_NAME] = df[TEXT_COLUMN_NAME].apply(lambda x: ' '.join([word for word in x if word not in stop_words]))

    return df

text_preprocessor = FunctionTransformer(preprocess_text)

In [22]:
def adapt_vectorizer_input(df: pd.DataFrame) -> Iterable:
    """Adapt input for the vectorizers.

    The problem is that vectorizers accept iterable, not DataFrame, but Series. Thus, we need to ravel dataframe with text have input single dimension.
    Issue reference: https://stackoverflow.com/questions/50665240/valueerror-found-input-variables-with-inconsistent-numbers-of-samples-1-3185"""

    df = df.iloc[:, 0]
    return df

vectorizer_input_adapter = FunctionTransformer(adapt_vectorizer_input)

## Define final pipeline

In [23]:
pipeline = Pipeline(steps=[
    ("text_preprocessor", text_preprocessor),
    ("vectorizer_input_adapter", vectorizer_input_adapter),
    ("vectorizer", CountVectorizer(ngram_range=(1, 1))),
    ("estimator", RandomForestClassifier(random_state=RANDOM_SEED))
])

## Fit and test estimator

In [24]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

                               precision    recall  f1-score   support

            Discharge Summary       0.43      0.38      0.41        34
         ENT - Otolaryngology       1.00      0.59      0.74        27
             Gastroenterology       0.57      0.81      0.67        43
             General Medicine       0.46      0.61      0.52        69
                    Neurology       0.74      0.66      0.70        53
                 Neurosurgery       0.73      0.79      0.76        24
      Obstetrics / Gynecology       0.85      0.58      0.69        50
SOAP / Chart / Progress Notes       0.40      0.36      0.38        33
                      Urology       0.74      0.68      0.71        38

                     accuracy                           0.61       371
                    macro avg       0.66      0.61      0.62       371
                 weighted avg       0.64      0.61      0.62       371



## Wrap model with giskard

In [25]:
wrapped_model = Model(pipeline,
                      model_type="classification",
                      name="medical_transcript_classification",
                      feature_names=[TEXT_COLUMN_NAME],
                      classification_labels=pipeline.classes_)

# Validate wrapped model and data.
print(classification_report(y_test, pipeline.classes_[wrapped_model.predict(wrapped_data).raw_prediction]))

                               precision    recall  f1-score   support

            Discharge Summary       0.43      0.38      0.41        34
         ENT - Otolaryngology       1.00      0.59      0.74        27
             Gastroenterology       0.57      0.81      0.67        43
             General Medicine       0.46      0.61      0.52        69
                    Neurology       0.74      0.66      0.70        53
                 Neurosurgery       0.73      0.79      0.76        24
      Obstetrics / Gynecology       0.85      0.58      0.69        50
SOAP / Chart / Progress Notes       0.40      0.36      0.38        33
                      Urology       0.74      0.68      0.71        38

                     accuracy                           0.61       371
                    macro avg       0.66      0.61      0.62       371
                 weighted avg       0.64      0.61      0.62       371



## Perform model scan

In [26]:
scanning_results = giskard.scan(wrapped_model, wrapped_data)

Your model is successfully validated.
Running scan…
Running detector ModelBiasDetector… 15 issues detected.
Running detector TextPerturbationDetector… 0 issues detected.
Scan completed: 15 issues found.


In [27]:
display(scanning_results)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">15 issues detected</span>
 </div>
 </div>

 
 <div data-tab-target="Model_bias" class="h-12 cursor-pointer group active">
 <div
 class="group-[.active]:bg-zinc-800 group-[.active]:border-b-zinc-800 group-[.active]:border-gray-500 border-l px-3 py-2 border-r border-t border-b border-gray-600 border-b-gray-500 h-11">
 Model bias
 
 
 <span class="ml-1 rounded-full text-xs min-w-4 min-h-4 px-1 py-0.5 inline-block text-center bg-red-400">
 
 15
 </span>
 
 </div>
 </div>
 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
<div id="Model_bias" role="tabpanel"
 class="dark:text-white dark:bg-zinc-800 p-4 pt-4 mb-4">
 
 <div class="p-3 mt-2 bg-amber-100/40 rounded-sm w-full flex align-middle">
 <div class="text-amber-100">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="mt-0.5 w-6 h-6">
 <path stroke-linecap="round" stroke-linejoin="round"
 d="M12 9v3.75m-9.303 3.376c-.866 1.5.217 3.374 1.948 3.374h14.71c1.73 0 2.813-1.874 1.948-3.374L13.949 3.378c-.866-1.5-3.032-1.5-3.898 0L2.697 16.126zM12 15.75h.007v.008H12v-.008z" />
 </svg>
 </div>
 <p class="ml-2 my-1 text-amber-100 text-sm">
 We detected some spots in your data where the model has a tendency to make incorrect
 predictions.
 </p>
 </div>
 

 
 <div class="flex items-center space-x-1">
 <h2 class="uppercase my-4 mr-2 font-medium">Issues</h2>
 
 <span class="text-xs border rounded px-1 uppercase text-red-400 border-red-400">15
 major</span>
 
 
 </div>

 <!-- ISSUES TABLE -->
<div class="mb-4">
 <table class="table-auto w-full text-white">
 
 <tbody class="first:border-t border-b border-zinc-400">
 <tr class="gsk-issue text-sm group peer text-left cursor-pointer hover:bg-zinc-700">
 <td class="p-3">
 <svg xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5"
 stroke="currentColor" class="w-4 h-4 group-[.open]:rotate-90">
 <path stroke-linecap="round" stroke-linejoin="round" d="M8.25 4.5l7.5 7.5-7.5 7.5" />
 </svg>
 </td>
 <td class="